### 2020301081 허종우 기말 프로젝트 Final Code
#### 이미지에 배경 효과를 입히고, 밝기와 명암도를 조절하는 프로그램

In [6]:
import cv2
import numpy as np

# 이미지 읽기
image = cv2.imread("image/bear.jpg")
if image is None:
    raise Exception("영상파일 읽기 오류")

# 프로젝트 주제는 이미지 배경효과 적용
title = 'Background Filtering'

# 초기 설정
current_filter = '0'
mode = None
scale = 0  # 트랙바 초기값
brightness = 50 # 밝기 초기값
contrast = 50   # 명암도 초기값

# 텍스트 표시 위치 계산 및 출력
line_spacing = 30  # 줄 간격
start_y = 30  # 첫 번째 줄의 Y 위치
x = 20  # X 위치

# 도움말 창 생성시 텍스트의 색상 적용을 위해 선언
white = (255, 255, 255)
black = (0, 0, 0)
navy = (128, 0, 0)

# 도움말 창 생성
hp_win = np.full((200, 450, 3), white, np.uint8)
hp_win2 = np.full((250, 400, 3), white, np.uint8)
font = cv2.FONT_HERSHEY_TRIPLEX;

First_Text = [
    "Press Buttons to adjust effects",
    "F. Filtering",
    "B. Brightness",
    "C. Contrast",
    "ESC. Exit Program & Save"
]

Second_Text = [
    "Filtering Effects",
    "1. Black & White",
    "2. WarmTone",
    "3. CoolTone",
    "4. Pencil Sketch",
    "5. Sharpening",
    "6. Cartoon",
    "0. Original",
    "ESC. Exit Program & Save"
]

# cv2.putText는 단일 줄만 출력가능, for 문 사용
for i, line in enumerate(First_Text):
    y = start_y + i * line_spacing  # Y 위치 계산
    if i == 0:
        color = navy
    else:
        color = black  # 첫 번째 줄만 강조 색상
    cv2.putText(hp_win, line, (x, y), font, 0.7, color, 1)

for i, line in enumerate(Second_Text):
    y = start_y + i * line_spacing  # Y 위치 계산
    if i == 0:
        color = navy
    else:
        color = black  # 첫 번째 줄만 강조 색상
    cv2.putText(hp_win2, line, (x, y), font, 0.7, color, 1)

# 밝기 및 명암도 조절 함수
def set_Brightness(val):
    global brightness
    brightness = val
    
def set_Contrast(val):
    global contrast
    contrast = val

def set_Scale(val):
    global scale
    scale = val
    
# 밝기 및 명암도 적용 함수
def btct(image):
    beta = brightness - 50  
    alpha = contrast / 50   
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

# 필터 변경 시 적용값 초기화
def reset_trackbar():
    global mode
    if mode:
        cv2.destroyWindow(mode)
        mode = None

# b, c 모드에서 숫자키를 누르면 비정상적인 종료 오류
def reset_scale():
    global scale
    if mode == "Scale":
        scale = 0
        cv2.setTrackbarPos(mode, "Scale", 0)  

def filtering(num, intensity):
    global filtered_image

    scale = intensity / 100  
    btct_image = btct(image)

    if num == '1':  # 흑백 변환
        gray = cv2.cvtColor(btct_image, cv2.COLOR_BGR2GRAY)
        gray_colored = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
        filtered_image = cv2.addWeighted(btct_image, 1 - scale, gray_colored, scale, 0)
    
    elif num == '2':  # 웜톤
        bgr = list(cv2.split(btct_image))
        bgr[2] = cv2.add(bgr[2], intensity)
        bgr[1] = cv2.add(bgr[1], intensity / 3)
        warmTone = cv2.merge(bgr)
        filtered_image = cv2.addWeighted(btct_image, 1 - scale, warmTone, scale, 0)
    
    elif num == '3':  # 쿨톤
        bgr = list(cv2.split(btct_image))
        bgr[0] = cv2.add(bgr[0], intensity)
        bgr[1] = cv2.add(bgr[1], intensity - 10)
        coolTone = cv2.merge(bgr)
        filtered_image = cv2.addWeighted(btct_image, 1 - scale, coolTone, scale, 0)

    elif num == '4':  # 연필 스케치 필터
        gray = cv2.cvtColor(btct_image, cv2.COLOR_BGR2GRAY)
        gray_invert = 255 - gray
        gaus_img = cv2.GaussianBlur(gray_invert, (35, 35), sigmaX=0, sigmaY=0)
        pencil = cv2.divide(gray, 255 - gaus_img, scale=255)
        pencil_colored = cv2.cvtColor(pencil, cv2.COLOR_GRAY2BGR)
        filtered_image = cv2.addWeighted(btct_image, 1 - scale, pencil_colored, scale, 0)

    elif num == '5':  # 샤프닝
        sharpen_kernel = np.array([[-1, -1, -1], 
                                   [-1,  9, -1], 
                                   [-1, -1, -1]])
        sharpened_image = cv2.filter2D(btct_image, -1, sharpen_kernel)
        filtered_image = cv2.addWeighted(btct_image, 1 - scale, sharpened_image, scale, 0)
        
    elif num == '6':  # 카툰화
        gray = cv2.cvtColor(btct_image, cv2.COLOR_BGR2GRAY)  # 그레이스케일 변환
        gray = cv2.GaussianBlur(gray, (9, 9), 0)  # 가우시안 필터 -> 잡음 제거
        edges = cv2.Laplacian(gray, -1, None, 5)  # 라플라시안 필터 -> 엣지 검출
        _, sketch = cv2.threshold(edges, 70, 255, cv2.THRESH_BINARY_INV)
        blurred = cv2.medianBlur(btct_image, 5)  # Median blur로 부드럽게 처리
        cartoon_image = cv2.bitwise_and(blurred, blurred, mask=sketch)
        filtered_image = cv2.addWeighted(btct_image, 1 - scale, cartoon_image, scale, 0)

    else:  # 필터가 없으면 원본 이미지 유지
        filtered_image = btct_image.copy()

cv2.namedWindow(title)
cv2.imshow('Help Window', hp_win)

while True:
    key = cv2.waitKeyEx(100)
    if key == 27:      # esc 누르면 종료
        break
    
    # 필터링 적용
    filtering(current_filter, scale)
    cv2.imshow(title, filtered_image)

    # 키보드 이벤트 처리
    if ord('0') <= key <= ord('6'):  # 필터 변경
        current_filter = chr(key)  # 현재 필터 업데이트
        reset_scale() 

    elif key == ord('f'):
        reset_trackbar()
        mode = "Scale"
        cv2.namedWindow(mode)
        cv2.imshow(mode, hp_win2)
        cv2.createTrackbar(mode, "Scale", scale, 100, set_Scale)

    # b, c가 눌린 상태에서 숫자키를 누르면 오류가 발생하는 예외 처리
    elif key == ord('b'):
        reset_trackbar()
        mode = "Brightness"
        cv2.namedWindow(mode, cv2.WINDOW_NORMAL)
        cv2.createTrackbar(mode, "Brightness", brightness, 100, set_Brightness)

    elif key == ord('c'):
        reset_trackbar()
        mode = "Contrast"
        cv2.namedWindow(mode, cv2.WINDOW_NORMAL)
        cv2.createTrackbar(mode, "Contrast", contrast, 100, set_Contrast)
   

# 편집본 이미지를 저장
save_path = "image/retouched.jpg"
cv2.imwrite(save_path, filtered_image)
print(f"편집본이 저장되었습니다: {save_path}")

cv2.destroyAllWindows()

편집본이 저장되었습니다: image/retouched.jpg
